In [1]:
import os
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer
from collections import defaultdict
import nltk
import numpy as np
import re
import math 
import pandas as pd

In [17]:
path_query = "../Queries/"

def natural_key(text):
    return [int(t) if t.isdigit() else t.lower()
            for t in re.split(r'(\d+)', text)]

queries = sorted(
    (f for f in os.listdir(path_query)
     if os.path.isfile(os.path.join(path_query, f))),
    key=natural_key
)

print(queries)

['query_1.txt', 'query_2.txt', 'query_3.txt', 'query_4.txt', 'query_5.txt', 'query_6.txt', 'query_7.txt', 'query_8.txt', 'query_9.txt', 'query_10.txt', 'query_11.txt', 'query_12.txt', 'query_13.txt', 'query_14.txt', 'query_15.txt', 'query_16.txt', 'query_17.txt', 'query_18.txt', 'query_19.txt', 'query_20.txt', 'query_21.txt', 'query_22.txt', 'query_23.txt', 'query_24.txt', 'query_25.txt', 'query_26.txt', 'query_27.txt', 'query_28.txt', 'query_29.txt', 'query_30.txt']


In [18]:
queries_list = []
for query in queries:
    with open(os.path.join(path_query, query), 'r', encoding='utf-8') as file:
        queries_list.append(file.read())

In [20]:
tokenizer = RegexpTokenizer(
    r'(?:[A-Za-z]\.)+'
    r'|[A-Za-z]+[\-@]\d+(?:\.\d+)?'
    r'|\d+(?:[\.\,\-]\d+)*%?'
    r'|[A-Za-z]+'
)

nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

In [24]:
def q_cleaning(query):
    tokens = tokenizer.tokenize(query)
    clean_query = [word.lower() for word in tokens if word.lower() not in stop_words]
    
    Porter = PorterStemmer()
    TermsPorter = [Porter.stem(t) for t in clean_query]
    
    return TermsPorter

def calc_weight_Word_query(query, word):
   
    if word in query:
        return 1
    
    return 0

In [25]:
clean_queries =[]

for q in queries_list:
    clean_queries.append(q_cleaning(q))

In [26]:
print(clean_queries[0])

['crystallin', 'len', 'vertebr', 'includ', 'human']


In [82]:
inverted_doc = pd.read_csv(
    "../DocTerm_matrix.txt",
    sep=r"\s+", header=None,
    names=["Document", "Term", "Frequency", "Weight"]
)
inverted_doc = inverted_doc.iloc[1:]

inverted_doc["Frequency"] = pd.to_numeric(inverted_doc["Frequency"], errors="coerce").fillna(0)
inverted_doc["Weight"] = pd.to_numeric(inverted_doc["Weight"], errors="coerce").fillna(0)

In [83]:
inverted_doc.head()

,Document,Term,Frequency,Weight
1,doc_1,acid,1,0.126638
2,doc_1,appear,1,0.107622
3,doc_1,coeffici,2,0.515906
4,doc_1,collect,1,0.217908
5,doc_1,correl,4,0.648133


In [84]:
duplicates_exist = inverted_doc.duplicated(subset=["Document", "Term"]).any()
print("Duplicates exist:", duplicates_exist)

Duplicates exist: False


In [85]:
M_weight = inverted_doc.pivot(index="Term", columns="Document", values="Weight").fillna(0)

In [86]:
M_freq = inverted_doc.pivot(index="Term", columns="Document", values="Frequency").fillna(0)

In [87]:
bin_mat = M_weight.map(lambda x: 1 if x > 0 else 0)

Parameters

In [89]:
lambda_ = 0.2
cf = M_freq.sum().sum()

In [90]:
print(cf)

96191.0


In [93]:
Navg = 0
for doc in bin_mat.columns:
    tf = M_freq[doc].sum()
    Navg += tf
Navg = Navg / 1033

In [94]:
print(Navg)

93.11810261374637


In [95]:
rsv_MLE, rsv_laplace, rsv_jelinek, rsv_dirichlet = {}, {}, {}, {}

In [109]:
for i, q in enumerate(clean_queries):
    rsv1 = {doc: 0.0 for doc in bin_mat.columns}  # log(1) = 0
    rsv2 = {doc: 0.0 for doc in bin_mat.columns}
    rsv3 = {doc: 0.0 for doc in bin_mat.columns}
    rsv4 = {doc: 0.0 for doc in bin_mat.columns}

    for term in q:
        if term not in M_freq.index:
            cfi = 0
        else:
            cfi = M_freq.loc[term].sum()

        for doc in bin_mat.columns:
            tfi = M_freq.loc[term, doc] if term in M_freq.index else 0
            tf = M_freq[doc].sum()

            # MLE
            if tfi > 0:
                rsv1[doc] += np.log(tfi / tf)
            # else: skip absent term

            # Laplace smoothing
            vocab_size = bin_mat[doc].sum()
            rsv2[doc] += np.log((tfi + 1) / (tf + vocab_size))

            # Jelinek-Mercer smoothing
            P_doc = tfi / tf if tf > 0 else 0
            P_col = cfi / cf if cf > 0 else 0
            score_jm = lambda_ * P_doc + (1 - lambda_) * P_col
            rsv3[doc] += np.log(score_jm + 1e-12)  # only tiny eps if score could be 0

            # Dirichlet smoothing
            mu = 0.3 * Navg
            score_dir = (tfi + mu * P_col) / (tf + mu)
            rsv4[doc] += np.log(score_dir + 1e-12)  # safe log

    # Save per query
    rsv_MLE[f"Query {i+1}"] = rsv1
    rsv_laplace[f"Query {i+1}"] = rsv2
    rsv_jelinek[f"Query {i+1}"] = rsv3
    rsv_dirichlet[f"Query {i+1}"] = rsv4


In [126]:
print(rsv_MLE)

{'Query 1': {'doc_1': 0.0, 'doc_10': 0.0, 'doc_100': np.float64(-5.313205979041787), 'doc_1000': 0.0, 'doc_1001': np.float64(-5.117993812416755), 'doc_1002': 0.0, 'doc_1003': 0.0, 'doc_1004': 0.0, 'doc_1005': 0.0, 'doc_1006': 0.0, 'doc_1007': 0.0, 'doc_1008': 0.0, 'doc_1009': 0.0, 'doc_101': 0.0, 'doc_1010': 0.0, 'doc_1011': 0.0, 'doc_1012': 0.0, 'doc_1013': 0.0, 'doc_1014': 0.0, 'doc_1015': 0.0, 'doc_1016': 0.0, 'doc_1017': 0.0, 'doc_1018': 0.0, 'doc_1019': 0.0, 'doc_102': 0.0, 'doc_1020': np.float64(-4.9344739331306915), 'doc_1021': 0.0, 'doc_1022': 0.0, 'doc_1023': 0.0, 'doc_1024': 0.0, 'doc_1025': 0.0, 'doc_1026': 0.0, 'doc_1027': 0.0, 'doc_1028': 0.0, 'doc_1029': np.float64(-4.859812404361672), 'doc_103': 0.0, 'doc_1030': 0.0, 'doc_1031': 0.0, 'doc_1032': 0.0, 'doc_1033': 0.0, 'doc_104': 0.0, 'doc_105': 0.0, 'doc_106': 0.0, 'doc_107': 0.0, 'doc_108': np.float64(-4.564348191467836), 'doc_109': 0.0, 'doc_11': np.float64(-3.0757749812275272), 'doc_110': 0.0, 'doc_111': 0.0, 'doc_112'

In [177]:
def save_rankings_to_csv(rsv_data, method_name, save_dir="rankings"):
    """
    Saves document rankings for each query into a CSV file.
    """
    # Create the directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Prepare data for a flat CSV structure
    rows = []
    for query_name, doc_scores in rsv_data.items():
        # Sort documents by score descending
        ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
        
        for rank, (doc, score) in enumerate(ranked_docs, 1):
            rows.append({
                "Query": query_name,
                "Rank": rank,
                "Document_ID": doc,
                "Log_Score": score
            })
    
    # Create DataFrame and save
    df = pd.DataFrame(rows)
    # Clean name for filename (remove spaces)
    clean_name = method_name.replace(" ", "_").lower()
    file_path = os.path.join(save_dir, f"ranking_{clean_name}.csv")
    
    df.to_csv(file_path, index=False)
    print(f"✅ Rankings for {method_name} saved to: {file_path}")

In [178]:
rsv_list = [rsv_MLE, rsv_laplace, rsv_jelinek, rsv_dirichlet]
method_names = ["MLE", "Laplace", "Jelinek-Mercer", "Dirichlet"]

for rsv, name in zip(rsv_list, method_names):
    save_rankings_to_csv(rsv, name)

✅ Rankings for MLE saved to: rankings\ranking_mle.csv
✅ Rankings for Laplace saved to: rankings\ranking_laplace.csv
✅ Rankings for Jelinek-Mercer saved to: rankings\ranking_jelinek-mercer.csv
✅ Rankings for Dirichlet saved to: rankings\ranking_dirichlet.csv


In [127]:
relevance = {}
relevance_folder = "../Relevances/"  # adjust path

for filename in os.listdir(relevance_folder):
    if filename.endswith(".txt"):
        query_name = filename.replace(".txt", "")  # e.g., "Query1"
        query_name = f"Query {query_name}"
        file_path = os.path.join(relevance_folder, filename)
        with open(file_path, "r") as f:
            docs = [f"doc_{line.strip()}" for line in f if line.strip()]
        relevance[query_name] = docs

In [128]:
print(relevance)

{'Query 1': ['doc_13', 'doc_14', 'doc_15', 'doc_72', 'doc_79', 'doc_138', 'doc_142', 'doc_164', 'doc_165', 'doc_166', 'doc_167', 'doc_168', 'doc_169', 'doc_170', 'doc_171', 'doc_172', 'doc_180', 'doc_181', 'doc_182', 'doc_183', 'doc_184', 'doc_185', 'doc_186', 'doc_211', 'doc_212', 'doc_499', 'doc_500', 'doc_501', 'doc_502', 'doc_503', 'doc_504', 'doc_506', 'doc_507', 'doc_508', 'doc_510', 'doc_511', 'doc_513'], 'Query 10': ['doc_54', 'doc_55', 'doc_58', 'doc_152', 'doc_153', 'doc_154', 'doc_155', 'doc_254', 'doc_255', 'doc_256', 'doc_257', 'doc_529', 'doc_531', 'doc_532', 'doc_533', 'doc_534', 'doc_535', 'doc_537', 'doc_538', 'doc_539', 'doc_540', 'doc_541', 'doc_542', 'doc_543'], 'Query 11': ['doc_32', 'doc_63', 'doc_66', 'doc_148', 'doc_150', 'doc_225', 'doc_226', 'doc_228', 'doc_229', 'doc_440', 'doc_441', 'doc_444', 'doc_445', 'doc_446', 'doc_447', 'doc_448', 'doc_451', 'doc_452'], 'Query 12': ['doc_16', 'doc_17', 'doc_19', 'doc_20', 'doc_193', 'doc_364', 'doc_365', 'doc_366', 'do

In [129]:
def display_rankings(rsv_data, method_name, relevance=None, top_k=None):
    print(f"\n{'='*60}")
    print(f"RANKING RESULTS: {method_name}")
    print(f"{'='*60}")

    for query_name, doc_scores in rsv_data.items():
        ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
        if top_k is not None:
            ranked_docs = ranked_docs[:top_k]

        print(f"\n🔹 {query_name}")
        print(f"{'Rank':<6} {'Document':<15} {'Log-Score':<12} {'Pertinent'}")
        print("-" * 60)
        for rank, (doc, score) in enumerate(ranked_docs, 1):
            pertinence = 0
            if relevance is not None:
                pertinence = 1 if doc in relevance.get(query_name, []) else 0
            print(f"{rank:<6} {doc:<15} {score:<12.6f} {pertinence}")
        break


In [130]:
display_rankings(rsv_MLE, "MLE", relevance=relevance)


RANKING RESULTS: MLE

🔹 Query 1
Rank   Document        Log-Score    Pertinent
------------------------------------------------------------
1      doc_1           0.000000     0
2      doc_10          0.000000     0
3      doc_1000        0.000000     0
4      doc_1002        0.000000     0
5      doc_1003        0.000000     0
6      doc_1004        0.000000     0
7      doc_1005        0.000000     0
8      doc_1006        0.000000     0
9      doc_1007        0.000000     0
10     doc_1008        0.000000     0
11     doc_1009        0.000000     0
12     doc_101         0.000000     0
13     doc_1010        0.000000     0
14     doc_1011        0.000000     0
15     doc_1012        0.000000     0
16     doc_1013        0.000000     0
17     doc_1014        0.000000     0
18     doc_1015        0.000000     0
19     doc_1016        0.000000     0
20     doc_1017        0.000000     0
21     doc_1018        0.000000     0
22     doc_1019        0.000000     0
23     doc_102         0

In [131]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve, average_precision_score

In [152]:
def overall_precision(ranked_docs, relevant_docs):
    tp = sum([1 for doc in ranked_docs if doc in relevant_docs])
    fp = len(ranked_docs) - tp
    return tp / (tp + fp) if (tp + fp) > 0 else 0

def overall_recall(ranked_docs, relevant_docs):
    tp = sum([1 for doc in ranked_docs if doc in relevant_docs])
    fn = len(relevant_docs) - tp
    return tp / (tp + fn) if (tp + fn) > 0 else 0

def overall_f1(ranked_docs, relevant_docs):
    p = overall_precision(ranked_docs, relevant_docs)
    r = overall_recall(ranked_docs, relevant_docs)
    return 2 * p * r / (p + r) if (p + r) > 0 else 0

# -----------------------------
# Top-K Metrics
# -----------------------------
def precision_at_k(ranked_docs, relevant_docs, k):
    ranked_docs = ranked_docs[:k]
    relevant_count = sum([1 for doc in ranked_docs if doc in relevant_docs])
    return relevant_count / k

# -----------------------------
# Overall Metrics
# -----------------------------
def r_precision(ranked_docs, relevant_docs):
    R = len(relevant_docs)
    return precision_at_k(ranked_docs, relevant_docs, R)

def reciprocal_rank(ranked_docs, relevant_docs):
    for idx, doc in enumerate(ranked_docs, 1):
        if doc in relevant_docs:
            return 1 / idx
    return 0

def dcg_at_k(ranked_docs, relevant_docs, k):
    dcg = 0
    for i, doc in enumerate(ranked_docs[:k], 1):
        if doc in relevant_docs:
            dcg += 1 / np.log2(i + 1)
    return dcg

def ndcg_at_k(ranked_docs, relevant_docs, k):
    ideal_dcg = dcg_at_k(sorted(relevant_docs, key=lambda x: 1, reverse=True), relevant_docs, k)
    actual_dcg = dcg_at_k(ranked_docs, relevant_docs, k)
    return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0

# -----------------------------
# Precision-Recall Curve
# -----------------------------
def precision_recall_curve(ranked_docs, relevant_docs):
    precisions, recalls = [], []
    n_relevant = len(relevant_docs)
    retrieved_relevant = 0

    for i, doc in enumerate(ranked_docs, 1):
        if doc in relevant_docs:
            retrieved_relevant += 1
        precisions.append(retrieved_relevant / i)
        recalls.append(retrieved_relevant / n_relevant if n_relevant else 0)
    return recalls, precisions

# -----------------------------
# Interpolated Precision-Recall Curve
# -----------------------------
def interpolated_precision_recall_curve(ranked_docs, relevant_docs):
    recalls, precisions = precision_recall_curve(ranked_docs, relevant_docs)
    interpolated_precisions = []
    for r in np.linspace(0, 1, 11):  # standard 11-point interpolation
        p = max([prec for rec, prec in zip(recalls, precisions) if rec >= r] + [0])
        interpolated_precisions.append(p)
    return np.linspace(0, 1, 11), interpolated_precisions

# -----------------------------
# Average Precision (AP) & MAP
# -----------------------------
def average_precision(ranked_docs, relevant_docs):
    precisions = []
    retrieved_relevant = 0
    for i, doc in enumerate(ranked_docs, 1):
        if doc in relevant_docs:
            retrieved_relevant += 1
            precisions.append(retrieved_relevant / i)
    return np.mean(precisions) if precisions else 0

def mean_average_precision(all_ranked_docs, all_relevant_docs):
    ap_scores = []
    for q in all_ranked_docs:
        ap_scores.append(average_precision(all_ranked_docs[q], all_relevant_docs[q]))
    return np.mean(ap_scores)

# -----------------------------
# Interpolated MAP
# -----------------------------
def interpolated_average_precision(ranked_docs, relevant_docs):
    recalls, precisions = precision_recall_curve(ranked_docs, relevant_docs)
    interpolated_precisions = []
    for r in np.linspace(0, 1, 11):
        p = max([prec for rec, prec in zip(recalls, precisions) if rec >= r] + [0])
        interpolated_precisions.append(p)
    return np.mean(interpolated_precisions)

def mean_interpolated_average_precision(all_ranked_docs, all_relevant_docs):
    scores = []
    for q in all_ranked_docs:
        scores.append(interpolated_average_precision(all_ranked_docs[q], all_relevant_docs[q]))
    return np.mean(scores)

# -----------------------------
# Gain (%) for nDCG@20
# -----------------------------
def gain_percent(all_ranked_docs, all_relevant_docs, k=20):
    gains = []
    for q in list(all_ranked_docs.keys())[:10]:  # first 10 queries
        ndcg = ndcg_at_k(all_ranked_docs[q], all_relevant_docs[q], k)
        gains.append(ndcg)
    return np.mean(gains) * 100

In [168]:
def evaluate_all_metrics(all_ranked_docs, all_relevant_docs, k_list=[5, 10], dcg_cutoff=20):

    metrics_dict = {}
    
    for query_name in all_ranked_docs:
        raw_results = all_ranked_docs[query_name]

        # If raw_results is a dict of {doc_id: score}, sort by score descending
        if isinstance(raw_results, dict):
            ranked_docs = sorted(raw_results, key=raw_results.get, reverse=True)
        else:
            ranked_docs = list(raw_results)
            
        relevant_docs = all_relevant_docs.get(query_name, [])

        metrics = {}
        
        # Overall Precision, Recall, F1
        metrics["Precision"] = overall_precision(ranked_docs, relevant_docs)
        metrics["Recall"] = overall_recall(ranked_docs, relevant_docs)
        metrics["F1"] = overall_f1(ranked_docs, relevant_docs)
        
        # Precision@K, Recall@K, F1@K
        for k in k_list:
            metrics[f"Precision@{k}"] = precision_at_k(ranked_docs, relevant_docs, k)
        
        # R-Precision
        metrics["R-Precision"] = r_precision(ranked_docs, relevant_docs)
        
        # Reciprocal Rank
        metrics["Reciprocal Rank"] = reciprocal_rank(ranked_docs, relevant_docs)
        
        # DCG and nDCG
        metrics[f"DCG@{dcg_cutoff}"] = dcg_at_k(ranked_docs, relevant_docs, dcg_cutoff)
        metrics[f"nDCG@{dcg_cutoff}"] = ndcg_at_k(ranked_docs, relevant_docs, dcg_cutoff)
        
        # Average Precision and Interpolated AP
        metrics["Average Precision"] = average_precision(ranked_docs, relevant_docs)
        metrics["Interpolated AP"] = interpolated_average_precision(ranked_docs, relevant_docs)
        
        # Precision-Recall Curve
        recalls, precisions = precision_recall_curve(ranked_docs, relevant_docs)
        metrics["Precision-Recall Curve"] = (recalls, precisions)
        
        # Interpolated Precision-Recall Curve
        interp_recalls, interp_precisions = interpolated_precision_recall_curve(ranked_docs, relevant_docs)
        metrics["Interpolated Precision-Recall Curve"] = (interp_recalls, interp_precisions)
        
        metrics_dict[query_name] = metrics
        
    # Overall metrics across all queries
    overall = {}
    overall["MAP"] = mean_average_precision(all_ranked_docs, all_relevant_docs)
    overall["Interpolated MAP"] = mean_interpolated_average_precision(all_ranked_docs, all_relevant_docs)
    overall["Gain (%)"] = gain_percent(all_ranked_docs, all_relevant_docs, k=dcg_cutoff)
    
    return metrics_dict, overall


In [170]:
import pandas as pd

def export_evaluation_report(query_metrics, overall_metrics, model_name):
    # --- FILE 1: Per-Query Metrics ---
    # Convert query metrics to DataFrame
    df_queries = pd.DataFrame.from_dict(query_metrics, orient='index')
    
    # Remove the PR Curve data (tuples) so the CSV remains readable/numeric
    cols_to_exclude = ["Precision-Recall Curve", "Interpolated Precision-Recall Curve"]
    df_queries_clean = df_queries.drop(columns=[c for c in cols_to_exclude if c in df_queries.columns])
    
    # Save Query-level results
    df_queries_clean.to_csv(f"metrics_per_query_{model_name}.csv", index_label="Query_ID")
    
    # --- FILE 2: System-Wide Summary ---
    # Convert overall metrics (MAP, etc.) to a vertical table
    df_overall = pd.DataFrame.from_dict(overall_metrics, orient='index', columns=['Value'])
    
    # Save Overall results
    df_overall.to_csv(f"metrics_overall_summary_{model_name}.csv", index_label="Metric")
    
    print("Files created: 'metrics_per_query.csv' and 'metrics_overall_summary.csv'")

In [173]:
import matplotlib.pyplot as plt
import numpy as np

def plot_pr_curves(query_metrics, model_name):
    # 1. Extract all Interpolated Precision arrays (11-point)
    # The data is stored in metrics["Interpolated Precision-Recall Curve"] as (recalls, precisions)
    all_interp_precisions = []
    
    for q in query_metrics:
        _, precs = query_metrics[q]["Interpolated Precision-Recall Curve"]
        all_interp_precisions.append(precs)
    
    # Calculate Mean Interpolated Precision across all queries
    mean_interp_precs = np.mean(all_interp_precisions, axis=0)
    recall_levels = np.linspace(0, 1, 11)

    # 2. Setup the Plot
    plt.figure(figsize=(10, 6))
    
    # Plot the 11-point Interpolated Curve (Step-like)
    plt.step(recall_levels, mean_interp_precs, where='post', label=f'Interpolated PR ({model_name})', linewidth=2, color='blue')
    plt.fill_between(recall_levels, mean_interp_precs, step='post', alpha=0.2, color='blue')

    # Formatting
    plt.title(f'11-Point Interpolated Precision-Recall Curve: {model_name}', fontsize=14)
    plt.xlabel('Recall', fontsize=12)
    plt.ylabel('Precision', fontsize=12)
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    
    file_path = f"PR_Curve_{model_name}.png"
    plt.savefig(file_path, dpi=300, bbox_inches='tight')
    plt.close() # Close the plot to free up memory
    
    print(f"Plot saved successfully at: {file_path}")

In [175]:
rsv_list = [rsv_MLE, rsv_laplace, rsv_jelinek, rsv_dirichlet]
method_names = ["MLE", "Laplace Smoothing", "Jelinek-Mercer Smoothing", "Dirichlet Smoothing"]
all_ranked = {}
for rsv, name in zip(rsv_list, method_names):
    ranked = {
        q_id: sorted(docs, key=docs.get, reverse=True) if isinstance(docs, dict) else list(docs)
        for q_id, docs in rsv.items()
    }
    query_metrics, overall_metrics = evaluate_all_metrics(ranked, relevance, k_list=[5, 10], dcg_cutoff=20)
    print(f"\n=== Evaluation Metrics for {name} ===")
    export_evaluation_report(query_metrics, overall_metrics, model_name=name)
    plot_pr_curves(query_metrics, name)
    all_ranked[name] = query_metrics


=== Evaluation Metrics for MLE ===
Files created: 'metrics_per_query.csv' and 'metrics_overall_summary.csv'
Plot saved successfully at: PR_Curve_MLE.png

=== Evaluation Metrics for Laplace Smoothing ===
Files created: 'metrics_per_query.csv' and 'metrics_overall_summary.csv'
Plot saved successfully at: PR_Curve_Laplace Smoothing.png

=== Evaluation Metrics for Jelinek-Mercer Smoothing ===
Files created: 'metrics_per_query.csv' and 'metrics_overall_summary.csv'
Plot saved successfully at: PR_Curve_Jelinek-Mercer Smoothing.png

=== Evaluation Metrics for Dirichlet Smoothing ===
Files created: 'metrics_per_query.csv' and 'metrics_overall_summary.csv'
Plot saved successfully at: PR_Curve_Dirichlet Smoothing.png


In [176]:
def compare_models_pr(all_model_results, output_folder="plots"):
    # Create folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    plt.figure(figsize=(10, 7))
    recall_levels = np.linspace(0, 1, 11)
    
    # Iterate through each model and plot its mean curve
    for model_name, query_metrics in all_model_results.items():
        # Collect the 11-point precision lists for all queries under this model
        all_precs = [
            query_metrics[q]["Interpolated Precision-Recall Curve"][1] 
            for q in query_metrics
        ]
        
        # Average them to get the system-wide performance
        mean_precs = np.mean(all_precs, axis=0)
        
        plt.plot(recall_levels, mean_precs, marker='o', label=model_name, linewidth=2)

    # Styling the comparison chart
    plt.title('System Comparison: 11-Point Interpolated Precision-Recall', fontsize=14)
    plt.xlabel('Recall', fontsize=12)
    plt.ylabel('Precision', fontsize=12)
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(loc='upper right')
    
    # Save the comparison image
    save_path = os.path.join(output_folder, "Model_Comparison_PR_Curve.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Comparison plot saved successfully at: {save_path}")

# Example Usage:
compare_models_pr(all_ranked)

Comparison plot saved successfully at: plots\Model_Comparison_PR_Curve.png
